In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Home.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [18]:
import http.client
import pandas as pd
import json
from datetime import datetime
import ssl
from django.db import IntegrityError, transaction
from basemodels.models import FikstureModelData


class FixtureDataHandler:

    def __init__(self, date):
        self.date = date

    def fetch_fixtures_from_api(self):
        """
        API'den fikstür verilerini çeker, hata durumunda boş değerlerle döner.
        """
        try:
            context = ssl._create_unverified_context()
            conn = http.client.HTTPSConnection('www.sofascore.com', context=context)
            conn.request('GET', f'/api/v1/sport/football/scheduled-events/{self.date}')
            response = conn.getresponse()
            print(response.status)
            data = json.loads(response.read())["events"]
            df = pd.DataFrame(data)
            model_data = [{
                'data_id': self.date.replace("-", ""),
                'tarih': self.date,
                'data': data,
                'count': len(df),
                'isprogress': False
            }]
        except Exception as e:
            print(f"API'den veri çekerken hata oluştu: {e}")
            model_data = [{
                'data_id': self.date.replace("-", ""),
                'tarih': self.date,
                'data': {},
                'count': 0,
                'isprogress': False
            }]
        
        return model_data

    def save_fixtures_to_db(self, api_data):
        """
        API'den gelen verileri veritabanına kaydeder. Mevcut kayıt varsa siler ve yenisini ekler.
        """
        for item in api_data:
            try:
                # Tarih formatını kontrol et ve hatalıysa varsayılan değeri kullan
                try:
                    tarih = datetime.strptime(item['tarih'], "%Y-%m-%d").date()
                except ValueError:
                    print(f"Geçersiz tarih formatı: {item['tarih']}, varsayılan tarih atanıyor.")
                    tarih = datetime.strptime("1900-01-01", "%Y-%m-%d").date()

                # Veritabanı işlemlerini atomik olarak başlat
                with transaction.atomic():
                    existing_entry = FikstureModelData.objects.filter(data_id=item['data_id']).first()

                    if existing_entry:
                        # Eğer veri mevcutsa, kaydı sil
                        existing_entry.delete()
                        print(f"Eski veri silindi: {item['data_id']}")

                    # Yeni kayıt ekle
                    FikstureModelData.objects.create(
                        data_id=item['data_id'],
                        tarih=tarih,  # Doğrulanmış tarih
                        data=item['data'],
                        count=item['count'],
                        isprogress=item['isprogress']
                    )
                    print(f"Yeni veri eklendi: {tarih}")

            except IntegrityError as e:
                print(f"IntegrityError oluştu: {e}")

                # Hata durumunda varsayılan değerlerle kayıt ekle
                with transaction.atomic():
                    FikstureModelData.objects.create(
                        data_id='19000101',  # Varsayılan data_id
                        tarih=datetime.strptime("1900-01-01", "%Y-%m-%d").date(),  # Varsayılan tarih
                        data={},  # Boş dictionary
                        count=0,  # Varsayılan 0
                        isprogress=False  # Varsayılan False
                    )
                    print(f"Boş değerlerle veri eklendi.")


# Örnek kullanım:
handler = FixtureDataHandler("2024-07-21")
data = handler.fetch_fixtures_from_api()
handler.save_fixtures_to_db(data)
print("İşlem Tamamlandı")
data

403
API'den veri çekerken hata oluştu: Expecting value: line 2 column 1 (char 1)
Eski veri silindi: 20240721
Yeni veri eklendi: 2024-07-21
İşlem Tamamlandı


[{'data_id': '20240721',
  'tarih': '2024-07-21',
  'data': {},
  'count': 0,
  'isprogress': False}]

In [5]:
from datetime import datetime, timedelta

# Başlangıç tarihi
start_date = datetime.strptime("2024-06-21", "%Y-%m-%d")
# Bitiş tarihi (bugünün tarihi)
end_date = datetime.strptime("2024-08-01", "%Y-%m-%d")

# Başlangıç ve bitiş tarihleri arasındaki gün sayısını hesapla
delta = end_date - start_date

# Her gün için döngü
for i in range(delta.days + 1):
    current_date = start_date + timedelta(days=i)
    date_str = current_date.strftime("%Y-%m-%d")
    
    # FixtureDataHandler'ı mevcut tarih ile oluştur
    handler = FixtureDataHandler(date_str)
    
    # API'den verileri çek
    data = handler.fetch_fixtures_from_api()
    
    # Verileri veritabanına kaydet
    handler.save_fixtures_to_db(data)
    
    print(f"{date_str} işlemi tamamlandı.")

print("Tüm işlemler tamamlandı.")


API'den veri çekerken hata oluştu: Expecting value: line 2 column 1 (char 1)
Eski veri silindi: 20240621
Yeni veri eklendi: 2024-06-21
2024-06-21 işlemi tamamlandı.
API'den veri çekerken hata oluştu: Expecting value: line 2 column 1 (char 1)
Eski veri silindi: 20240622
Yeni veri eklendi: 2024-06-22
2024-06-22 işlemi tamamlandı.
API'den veri çekerken hata oluştu: Expecting value: line 2 column 1 (char 1)
Eski veri silindi: 20240623
Yeni veri eklendi: 2024-06-23
2024-06-23 işlemi tamamlandı.
API'den veri çekerken hata oluştu: Expecting value: line 2 column 1 (char 1)
Eski veri silindi: 20240624
Yeni veri eklendi: 2024-06-24
2024-06-24 işlemi tamamlandı.
API'den veri çekerken hata oluştu: Expecting value: line 2 column 1 (char 1)
Eski veri silindi: 20240625
Yeni veri eklendi: 2024-06-25
2024-06-25 işlemi tamamlandı.
API'den veri çekerken hata oluştu: Expecting value: line 2 column 1 (char 1)
Eski veri silindi: 20240626
Yeni veri eklendi: 2024-06-26
2024-06-26 işlemi tamamlandı.
API'den ve